# TigerGraph Data Science Library 101 - Classification Algorithm
This notebook shows the examples of using the most common classification algorithms in TigerGraph Graph Science Library. More detailed explanations of these algorithms can be found in the official documentation (https://docs.tigergraph.com/graph-ml/current/classification-algorithms/).


## Step1: Setting things up
- Connect and Load data
- Visualize the graph schema 
- Get basic stats, e.g., counts of nodes & edges

### Create connection

In [1]:
import json
import pandas as pd
from pyTigerGraph import TigerGraphConnection

# Read in DB configs
with open('../config.json', "r") as config_file:
    config = json.load(config_file)

conn = TigerGraphConnection(
    host=config["host"],
    username=config["username"],
    password=config["password"],
)

### Download movie dataset

In [2]:
from pyTigerGraph.datasets import Datasets

dataset_movie = Datasets("movie")

Downloading:   0%|          | 0/2623 [00:00<?, ?it/s]

### Ingest data

In [3]:
from pyTigerGraph.visualization import drawSchema

conn.ingestDataset(dataset_movie, getToken=config["getToken"])

---- Checking database ----
---- Creating graph ----
The graph movie is created.
---- Creating schema ----
Using graph 'movie'
Successfully created schema change jobs: [movie_schema].
Kick off schema change job movie_schema
Doing schema change on graph 'movie' (current version: 0)
Trying to add local vertex 'Person' to the graph 'movie'.
Trying to add local vertex 'Movie' to the graph 'movie'.
Trying to add local edge 'Likes' and its reverse edge 'reverse_Likes' to the graph 'movie'.
Trying to add local edge 'Similarity' to the graph 'movie'.

Graph movie updated to new version 1
The job movie_schema completes in 1.955 seconds!
---- Creating loading job ----
Using graph 'movie'
Successfully created loading jobs: [load_movie].
---- Ingesting data ----
Ingested 16 objects into VERTEX Person
Ingested 16 objects into VERTEX Movie
Ingested 15 objects into EDGE Likes
---- Cleaning ----
---- Finished ingestion ----


### Visualize schema

In [4]:
drawSchema(conn.getSchema(force=True))

CytoscapeWidget(cytoscape_layout={'name': 'circle', 'animate': True, 'padding': 1}, cytoscape_style=[{'selecto…

### Print graph stats

In [5]:
vertices = conn.getVertexTypes()
total_count = 0
for vertex in vertices:
    vertex_cnt = conn.getVertexCount(vertex)
    total_count += vertex_cnt
    print("Node count: ({} : {}) ".format(vertex, vertex_cnt))
print("Total node count: ", total_count)

Node count: (Person : 7) 
Node count: (Movie : 9) 
Total node count:  16


In [6]:
import pprint
edge_count = conn.getEdgeCount()
print("Edges count: total ", sum(edge_count.values()))
pprint.pprint(edge_count) 

Edges count: total  30
{'Likes': 15, 'Similarity': 0, 'reverse_Likes': 15}


## Step 2: Leveraging pyTigerGraph’s featurizer to run Classification algorithms

pyTigerGraph provides a full suit of data science capabilities, and in this tutorial, we will showcase how to use featurizer to list out all available Classification algorithms in our GDS library, and to run a few popular algorithms as an example.

In [7]:
feat = conn.gds.featurizer()

In [8]:
feat.listAlgorithms("Classification")

Available algorithms for Classification:
  greedy_graph_coloring:
    01. name: tg_greedy_graph_coloring
  k_nearest_neighbors:
    all_pairs:
      02. name: tg_knn_cosine_all
    cross_validation:
      03. name: tg_knn_cosine_cv
    single_source:
      04. name: tg_knn_cosine_ss
  maximal_independent_set:
    deterministic:
      05. name: tg_maximal_indep_set
    random:
      06. name: tg_maximal_indep_set_random
Call runAlgorithm() with the algorithm name to execute it


## tg_knn_cosine_ss

The k-Nearest Neighbors (kNN) algorithm is one of the simplest classification algorithms. It assumes that some or all the vertices in the graph have already been classified. The classification is stored as an attribute called the label. The goal is to predict the label of a given vertex, by seeing what are the labels of the nearest vertices.

Given a source vertex in the dataset and a positive integer k, the algorithm calculates the distance between this vertex and all other vertices and selects the k vertices that are nearest. The prediction of the label of this node is the majority label among its k-nearest neighbors. (https://docs.tigergraph.com/graph-ml/current/classification-algorithms/k-nearest-neighbors)

## Input Parameters

* VERTEX source: The vertex which you want to predict the label {"id": "vertex_id", "type": "vertex_type"}
* SET<STRING> v_type_set: Vertex types to calculate distance to source vertex for
* SET<STRING> e_type_set: Edge types to traverse
* SET<STRING> reverse_e_type_set: Reverse edge types to traverse
* STRING weight_attribute: Edge attribute to use as the weight of the edge
* STRING label: Vertex attribute to recognize as the label of the vertex
* INT top_k: number of nearest neighbors to consider
* BOOL print_results: Boolean value that indicates whether to output to console in JSON
* STRING filepath: If provided, the algorithm will output to this file path in CSV format
* STRING result_attribute: Vertex attribute to save the predicted label as.

In [9]:
params = {
    "source": {"id": "Neil", "type": "Person"},
    "v_type_set": ["Person"],
    "e_type_set": ["Likes"],
    "reverse_e_type_set": ["reverse_Likes"],
    "weight_attribute": "weight",
    "label": "known_label",
    "top_k": 5,
    "print_results": True,
    "file_path": "",
    "result_attribute": "predicted_label"
}

In [10]:
import csv
import os
import time
import psutil
!pip install memory_profiler
%load_ext memory_profiler

algo_performance_out = '/home/tigergraph/GraphML/output/algorithm_' + config["job_id"] + '.csv'

start_time = time.time()

algo_memory = %memit -r 1 -o feat.runAlgorithm("tg_knn_cosine_ss", params=params)

algo_memory = str(algo_memory)

start = algo_memory.find(": ") + 1
end = algo_memory.find("M")

algo_memory = algo_memory[start:end].strip()

execution_time = time.time() - start_time

cpu_usage = psutil.cpu_percent(4)

print('The CPU usage is: ', cpu_usage)

# print('RAM memory % used:', psutil.virtual_memory()[2])

host_memory = psutil.virtual_memory()[3]/1000000000

print('RAM Used (GB):', host_memory)

print ('tg_knn_cosine_ss executed successfully')

print ('execution time: ' + str(execution_time) + ' seconds\n')

algo_id = "tg_knn_cosine_ss_" + config["job_id"]

nb_id = "algos_classification.ipynb_" + config["job_id"]

data = [algo_id, "false" ,cpu_usage, algo_memory, execution_time, host_memory, "3.8", "no error", nb_id]

with open(algo_performance_out, mode='a+', encoding='utf-8') as f:
    writer = csv.writer(f) 
    writer.writerow(data)

/opt/conda/lib/python3.9/site-packages/memory_profiler.py:1136: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ipython_version = LooseVersion(IPython.__version__)
/opt/conda/lib/python3.9/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


Installing and optimizing the queries, it might take a minute...
Queries installed successfully
peak memory: 123.75 MiB, increment: 1.43 MiB
The CPU usage is:  22.5
RAM Used (GB): 10.211319808
tg_knn_cosine_ss executed successfully
execution time: 67.69653797149658 seconds



## Results 

We then run kNN, using Neil as the source person and k=3, the persons with the top 3 similarity score: Kat: 0.67509, Jing: 0.46377, Kevin: 0.42436, both of Kat and Kevin have label a.

In [12]:
results = feat.runAlgorithm("tg_knn_cosine_ss", params=params)
df_knn_cosine_ss = pd.json_normalize(results)
display(df_knn_cosine_ss)

,predicted_label
0,a


## tg_knn_cosine_all

This algorithm is a batch version of the k-Nearest Neighbors, Cosine Neighbor Similarity, single vertex. It makes a prediction for every vertex whose label is not known (i.e., the attribute for the known label is empty), based on its k nearest neighbors' labels.(https://docs.tigergraph.com/graph-ml/current/classification-algorithms/k-nearest-neighbors-batch)

## Input Parameters

* SET<STRING> v_type_set: Vertex types to calculate distance to source vertex for
* SET<STRING> e_type_set: Edge types to traverse
* SET<STRING> reverse_e_type_set: Reverse edge types to traverse
* STRING weight_attribute: Edge attribute to use as the weight of the edge
* STRING label: Vertex attribute to recognize as the label of the vertex
* INT top_k: number of nearest neighbors to consider
* BOOL print_results: Boolean value that indicates whether to output to console in JSON
* STRING filepath: If provided, the algorithm will output to this file path in CSV format
* STRING result_attribute: Vertex attribute to save the predicted label as.

In [13]:
params = {
    "v_type_set": ["Person"],
    "e_type_set": ["Likes"],
    "reverse_e_type_set": ["reverse_Likes"],
    "weight_attribute": "weight",
    "label": "known_label",
    "top_k": 3,
    "print_results": True,
    "file_path": "",
    "result_attribute": "predicted_label"
}

In [14]:
start_time = time.time()

algo_memory = %memit -r 1 -o feat.runAlgorithm("tg_knn_cosine_all", params=params)

algo_memory = str(algo_memory)

start = algo_memory.find(": ") + 1
end = algo_memory.find("M")

algo_memory = algo_memory[start:end].strip()

execution_time = time.time() - start_time

cpu_usage = psutil.cpu_percent(4)

print('The CPU usage is: ', cpu_usage)

# print('RAM memory % used:', psutil.virtual_memory()[2])

host_memory = psutil.virtual_memory()[3]/1000000000

print('RAM Used (GB):', host_memory)

print ('tg_knn_cosine_all executed successfully')

print ('execution time: ' + str(execution_time) + ' seconds\n')

algo_id = "tg_knn_cosine_all_" + config["job_id"]

nb_id = "algos_classification.ipynb_" + config["job_id"]

data = [algo_id, "false" ,cpu_usage, algo_memory, execution_time, host_memory, "3.8", "no error", nb_id]

with open(algo_performance_out, mode='a+', encoding='utf-8') as f:
    writer = csv.writer(f) 
    writer.writerow(data)

Installing and optimizing the queries, it might take a minute...
Queries installed successfully
Installing and optimizing the queries, it might take a minute...
Queries installed successfully
peak memory: 130.36 MiB, increment: 0.01 MiB
The CPU usage is:  27.3
RAM Used (GB): 10.373251072
tg_knn_cosine_all executed successfully
execution time: 71.12976360321045 seconds



## Results 

The predicted label for the vertices whose label attribute is empty.

For the movie graph shown in the single vertex version, run knn_cosine_all, using topK=3. Then you get the following result:

In [15]:
results = feat.runAlgorithm("tg_knn_cosine_all", params=params)
df_knn_cosine_all = pd.json_normalize(results, record_path =['source'])
display(df_knn_cosine_all)

,v_id,v_type,attributes.name,attributes.known_label,attributes.predicted_label,attributes.@sum_predicted_label
0,Jing,Person,Jing,,a,a
1,Neil,Person,Neil,,b,b
2,Elena,Person,Elena,,,


## tg_knn_cosine_cv

k-Nearest Neighbors (kNN) is often used for machine learning. You can choose the value for topK based on your experience, or using cross-validation to optimize the hyperparameters. In our library, Leave-one-out cross-validation for selecting optimal k is provided. Given a k value, we run the algorithm repeatedly using every vertex with a known label as the source vertex and predict its label. We assess the accuracy of the predictions for each value of k, and then repeat for different values of k in the given range. The goal is to find the value of k with highest predicting accuracy in the given range, for that dataset. (https://docs.tigergraph.com/graph-ml/current/classification-algorithms/k-nearest-neighbors-cross-validation)

## Input Parameters

* SET<STRING> v_type_set: Vertex types to calculate distance to source vertex for
* SET<STRING> e_type_set: Edge types to traverse
* SET<STRING> reverse_e_type_set: Reverse edge types to traverse
* STRING weight_attribute: Edge attribute to use as the weight of the edge
* STRING label: Vertex attribute to recognize as the label of the vertex
* INT min_k: lower bound of k (inclusive)
* INT max_k: upper bound of k (inclusive)

In [16]:
params = {
    "v_type_set": ["Person"],
    "e_type_set": ["Likes"],
    "reverse_e_type_set": ["reverse_Likes"],
    "weight_attribute": "weight",
    "label": "known_label",
    "min_k": 2,
    "max_k": 5
}

In [17]:
start_time = time.time()

algo_memory = %memit -r 1 -o feat.runAlgorithm("tg_knn_cosine_cv", params=params)

algo_memory = str(algo_memory)

start = algo_memory.find(": ") + 1
end = algo_memory.find("M")

algo_memory = algo_memory[start:end].strip()

execution_time = time.time() - start_time

cpu_usage = psutil.cpu_percent(4)

print('The CPU usage is: ', cpu_usage)

# print('RAM memory % used:', psutil.virtual_memory()[2])

host_memory = psutil.virtual_memory()[3]/1000000000

print('RAM Used (GB):', host_memory)

print ('tg_knn_cosine_cv executed successfully')

print ('execution time: ' + str(execution_time) + ' seconds\n')

algo_id = "tg_knn_cosine_cv_" + config["job_id"]

nb_id = "algos_classification.ipynb_" + config["job_id"]

data = [algo_id, "false" ,cpu_usage, algo_memory, execution_time, host_memory, "3.8", "no error", nb_id]

with open(algo_performance_out, mode='a+', encoding='utf-8') as f:
    writer = csv.writer(f) 
    writer.writerow(data)

Installing and optimizing the queries, it might take a minute...
Queries installed successfully
Installing and optimizing the queries, it might take a minute...
Queries installed successfully
peak memory: 130.49 MiB, increment: 0.09 MiB
The CPU usage is:  23.9
RAM Used (GB): 10.3796736
tg_knn_cosine_cv executed successfully
execution time: 74.10642075538635 seconds



## Results 

A list of prediction accuracy for every k value in the given range, and the value of k with the highest predicting accuracy in the given range.

Run knn_cosine_cv with min_k=2, max_k = 5.

In [19]:
results = feat.runAlgorithm("tg_knn_cosine_cv", params=params)
y = json.dumps(results, indent = 1)
print (y)

[
 {
  "@@correct_rate_list": [
   0.25,
   0.25,
   0.25,
   0.25
  ]
 },
 {
  "best_k": 2
 }
]


## Switch to scoial graph to showcase the rest of algorithms

* Connect and Load data
* Visualize the graph schema 
* Get basic stats, e.g., counts of nodes & edges


### Download scocial dataset

In [20]:
dataset_social = Datasets("social")

Downloading:   0%|          | 0/1970 [00:00<?, ?it/s]

### Ingest data

In [21]:
conn.ingestDataset(dataset_social, getToken=config["getToken"])

---- Checking database ----
---- Creating graph ----
The graph social is created.
---- Creating schema ----
Using graph 'social'
Successfully created schema change jobs: [social_schema].
Kick off schema change job social_schema
Doing schema change on graph 'social' (current version: 0)
Trying to add local vertex 'Person' to the graph 'social'.
Trying to add local edge 'Friend' and its reverse edge 'reverse_Friend' to the graph 'social'.
Trying to add local edge 'Coworker' to the graph 'social'.

Graph social updated to new version 1
The job social_schema completes in 2.657 seconds!
---- Creating loading job ----
Using graph 'social'
Successfully created loading jobs: [load_social].
---- Ingesting data ----
Ingested 17 objects into VERTEX Person
Ingested 15 objects into VERTEX Person
Ingested 14 objects into EDGE Friend
Ingested 13 objects into EDGE Coworker
---- Cleaning ----
---- Finished ingestion ----


### Connect to graph

In [22]:
# if graph alreay exit (data injection was done in the past) 
conn.graphname = "social"
if config["getToken"]:  
    conn.getToken(conn.createSecret())

### Visualize schema

In [23]:
drawSchema(conn.getSchema(force=True))

CytoscapeWidget(cytoscape_layout={'name': 'circle', 'animate': True, 'padding': 1}, cytoscape_style=[{'selecto…

### Print graph stats

In [24]:
vertices = conn.getVertexTypes()
total_count = 0
for vertex in vertices:
    vertex_cnt = conn.getVertexCount(vertex)
    total_count += vertex_cnt
    print("Node count: ({} : {}) ".format(vertex, vertex_cnt))
print("Total node count: ", total_count)

Node count: (Person : 12) 
Total node count:  12


In [25]:
import pprint
edge_count = conn.getEdgeCount()
print("Edges count: total ", sum(edge_count.values()))
pprint.pprint(edge_count) 

Edges count: total  39
{'Coworker': 11, 'Friend': 14, 'reverse_Friend': 14}


## tg_maximal_indep_set

An independent set of vertices does not contain any pair of vertices that are neighbors, i.e., ones which have an edge between them. A maximal independent set (MIS) is the largest independent set that contains those vertices; you cannot improve upon it unless you start over with a different independent set. However, the search for the largest possible independent set is an NP-hard problem: there is no known algorithm that can find that answer in polynomial time. So we settle for the maximal independent set.

This algorithm finds use in applications wanting to find the most efficient configuration which "covers" all the necessary cases. For example, it has been used to optimize delivery or transit routes, where each vertex is one transit segment and each edge connects two segments that can not be covered by the same vehicle.

Since there could be multiple maximal independent sets, there are two versions of the Maximal Independent Set algorithm:

Deterministic. The deterministic version makes sure that you get the same results every time. (https://docs.tigergraph.com/graph-ml/current/classification-algorithms/maximal-independent-set)

## Input Parameters

* STRING v_type: Name of vertex type to use
* STRING e_type: Name of edge type to use
* INT maximum_iteration: maximum number of iterations for the search
* BOOL print_results: If True, output JSON to standard output
* STRING file_path: If not empty, write output to this file.

In [26]:
params = {
    "v_type": "Person",
    "e_type": "Coworker",
    "maximum_iteration": 100,
    "print_results": True,
    "file_path": ""
}

In [27]:
start_time = time.time()

algo_memory = %memit -r 1 -o feat.runAlgorithm("tg_maximal_indep_set", params=params)

algo_memory = str(algo_memory)

start = algo_memory.find(": ") + 1
end = algo_memory.find("M")

algo_memory = algo_memory[start:end].strip()

execution_time = time.time() - start_time

cpu_usage = psutil.cpu_percent(4)

print('The CPU usage is: ', cpu_usage)

# print('RAM memory % used:', psutil.virtual_memory()[2])

host_memory = psutil.virtual_memory()[3]/1000000000

print('RAM Used (GB):', host_memory)

print ('tg_maximal_indep_set executed successfully')

print ('execution time: ' + str(execution_time) + ' seconds\n')

algo_id = "tg_maximal_indep_set_" + config["job_id"]

nb_id = "algos_classification.ipynb_" + config["job_id"]

data = [algo_id, "false" ,cpu_usage, algo_memory, execution_time, host_memory, "3.8", "no error", nb_id]

with open(algo_performance_out, mode='a+', encoding='utf-8') as f:
    writer = csv.writer(f) 
    writer.writerow(data)

Installing and optimizing the queries, it might take a minute...
Queries installed successfully
peak memory: 130.62 MiB, increment: 0.10 MiB
The CPU usage is:  24.7
RAM Used (GB): 10.24616448
tg_maximal_indep_set executed successfully
execution time: 65.21586489677429 seconds



## Results

A set of vertices that form a maximal independent set.

In [28]:
results = feat.runAlgorithm("tg_maximal_indep_set", params=params)
df_maximal_indep_set = pd.json_normalize(results, record_path =['Start'])
display(df_maximal_indep_set)

,v_id,v_type,attributes.name,attributes.score,attributes.tag,attributes.flag,attributes.@and_active,attributes.@or_selected,attributes.@min_vid
0,Eddie,Person,Eddie,0,,False,False,True,347078656
1,Justin,Person,Justin,0,,False,False,True,373293056
2,dirTarget,Person,dirTarget,0,,False,False,True,9223372036854775807
3,Ivy,Person,Ivy,0,,False,False,True,369098752
4,source,Person,source,0,,False,False,True,9223372036854775807


## tg_greedy_graph_coloring
This algorithm assigns a unique integer value known as its color to the vertices of a graph such that no neighboring vertices share the same color. The reason why this is called color is that this task is equivalent to assigning a color to each nation on a map so that no neighboring nations share the same color. (https://docs.tigergraph.com/graph-ml/current/classification-algorithms/greedy-graph-coloring)

## Input Parameters

* SET<STRING> v_type_set: A set of all vertex types to color.
* SET<STRING> e_type_set: A set of all edge types to traverse.
* UINT max_colors: The Maximum number of colors that can be used. Use a large number like 999999 unless there is a strict limit.
* BOOL print_color_count: If set to true, the total number of colors used will be displayed
* BOOL print_stats: If set to true, the output will display all vertices and their associated color
* STRING file_path: If a file path is provided, the output will be saved to the file indicated by the file path in CSV format.

In [29]:
params = {
    "v_type_set": ["Person"],
    "e_type_set": ["Friend", "Coworker"],
    "max_colors": 999999,
    "print_color_count": True,
    "print_stats": True,
    "file_path": ""
}

In [30]:
start_time = time.time()

algo_memory = %memit -r 1 -o feat.runAlgorithm("tg_greedy_graph_coloring", params=params)

algo_memory = str(algo_memory)

start = algo_memory.find(": ") + 1
end = algo_memory.find("M")

algo_memory = algo_memory[start:end].strip()

execution_time = time.time() - start_time

cpu_usage = psutil.cpu_percent(4)

print('The CPU usage is: ', cpu_usage)

# print('RAM memory % used:', psutil.virtual_memory()[2])

host_memory = psutil.virtual_memory()[3]/1000000000

print('RAM Used (GB):', host_memory)

print ('tg_greedy_graph_coloring executed successfully')

print ('execution time: ' + str(execution_time) + ' seconds\n')

algo_id = "tg_greedy_graph_coloring_" + config["job_id"]

nb_id = "algos_classification.ipynb_" + config["job_id"]

data = [algo_id, "false" ,cpu_usage, algo_memory, execution_time, host_memory, "3.8", "no error", nb_id]

with open(algo_performance_out, mode='a+', encoding='utf-8') as f:
    writer = csv.writer(f) 
    writer.writerow(data)

Installing and optimizing the queries, it might take a minute...
Queries installed successfully
peak memory: 130.72 MiB, increment: 0.04 MiB
The CPU usage is:  24.8
RAM Used (GB): 10.425483264
tg_greedy_graph_coloring executed successfully
execution time: 39.27679681777954 seconds



## Results

On the social graph, we want to color the Person vertices and any two vertices are either connected by a Friend edge or a Coworker edge do not have the same color. By running the greedy_graph_color algorithm, we get the following result:

In [32]:
results = feat.runAlgorithm("tg_greedy_graph_coloring", params=params)
r = json.dumps(results, indent = 1)
print (r)

[
 {
  "color_count": 4
 },
 {
  "start": [
   {
    "v_id": "Eddie",
    "v_type": "Person",
    "attributes": {
     "start.@sum_color_vertex": 3
    }
   },
   {
    "v_id": "Ivy",
    "v_type": "Person",
    "attributes": {
     "start.@sum_color_vertex": 4
    }
   },
   {
    "v_id": "Justin",
    "v_type": "Person",
    "attributes": {
     "start.@sum_color_vertex": 4
    }
   },
   {
    "v_id": "Damon",
    "v_type": "Person",
    "attributes": {
     "start.@sum_color_vertex": 2
    }
   },
   {
    "v_id": "Bob",
    "v_type": "Person",
    "attributes": {
     "start.@sum_color_vertex": 2
    }
   },
   {
    "v_id": "George",
    "v_type": "Person",
    "attributes": {
     "start.@sum_color_vertex": 2
    }
   },
   {
    "v_id": "Howard",
    "v_type": "Person",
    "attributes": {
     "start.@sum_color_vertex": 1
    }
   },
   {
    "v_id": "Alex",
    "v_type": "Person",
    "attributes": {
     "start.@sum_color_vertex": 1
    }
   },
   {
    "v_id": "Fiona",
    